# Semantic LLM caching

## Colab-specific setup

Make sure you have a DB, a bundle and a token. See [BD setup](https://cassio.org/db_setup/) on cassio.org for details.
Get ready to upload the bundle and supply the token string.

Likewise, ensure you have the necessary secret for a LLM provider: you'll be asked to input it shortly. See [Api setup](https://cassio.org/api_setup/) on cassio.org for details.

_Note: this colab is autogenerated from a regular Jupyter notebook hosted by cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "git+https://github.com/datastax/python-driver.git@cep-vsearch#egg=cassandra-driver" \
    "cassio>=0.0.2" \
    "feast>=0.26" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
    ExecutionProfile,
    EXEC_PROFILE_DEFAULT,
    ConsistencyLevel,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        profile = ExecutionProfile(
            consistency_level=ConsistencyLevel.LOCAL_QUORUM,
        )
        #
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
            execution_profiles={EXEC_PROFILE_DEFAULT: profile},
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unknown CQL Session mode')

# Semantic LLM caching

_**NOTE:** this uses Cassandra's experimental "Vector Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

The Cassandra-backed "semantic cache" for prompt responses is imported like this:

In [1]:
from langchain.cache import CassandraSemanticCache

As usual, a database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Cassandra cluster_ is running locally. Adjust as needed.

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

An embedding function and an LLM are needed.

Below is the logic to instantiate the LLM and embeddings of choice. We choose to leave it in the notebooks for clarity.

In [3]:
# creation of the LLM resources


if llmProvider == 'VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


**Note**: for the time being you have to explicitly _turn on this experimental flag_ on the `cassio` side:

In [4]:
import cassio
cassio.globals.enableExperimentalVectorSearch()

## Create the cache

At this point you can instantiate the semantic cache.

_Note: in the following it is made clear, through the `table_name_prefix` parameter, that different embeddings will require separate tables. This is done here to avoid mismatches when running this demo over and over with varying embedding functions: in most applications, where a single embedding suffices, there's no need to be this finicky._

In [5]:
cassSemanticCache = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

Make sure the cache starts empty with:

In [6]:
cassSemanticCache.clear_through_llm(llm=llm)

Configure the cache at a LangChain global level:

In [7]:
import langchain
langchain.llm_cache = cassSemanticCache

## Use the cache

Now try submitting a few prompts to the LLM and pay attention to the response times.

If the LLM is actually run, they should be the order of a few seconds; but in case of a cache hit, it will be way less than a second.

Notice that you get a cache hit even after rephrasing the question.

In [8]:
%%time
SPIDER_QUESTION_FORM_1 = "How many eyes do spiders have?"
# A new question should take long
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 13 ms, sys: 0 ns, total: 13 ms
Wall time: 2.12 s


'\n\nMost spiders have eight eyes.'

In [9]:
%%time
# Second time, very same question, this should be quick
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 5.57 ms, sys: 1.89 ms, total: 7.46 ms
Wall time: 130 ms


'\n\nMost spiders have eight eyes.'

In [10]:
%%time
SPIDER_QUESTION_FORM_2 = "How many eyes does a spider generally have?"
# Just a rephrasing: but it's the same question,
# so it will just take the time to evaluate embeddings
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 9.9 ms, sys: 0 ns, total: 9.9 ms
Wall time: 343 ms


'\n\nMost spiders have eight eyes.'

Time for a really new question:

In [11]:
%%time
LOGIC_QUESTION_FORM_1 = "Is absence of proof the same as proof of absence?"
# A totally new question
llm(LOGIC_QUESTION_FORM_1)

CPU times: user 9.92 ms, sys: 3.8 ms, total: 13.7 ms
Wall time: 1.75 s


'\n\nNo, absence of proof is not the same as proof of absence.'

In [12]:
%%time
SPIDER_QUESTION_FORM_3 = "How many eyes are on the head of a typical spider?"
# Trying to catch the cache off-guard :)
llm(SPIDER_QUESTION_FORM_3)

CPU times: user 8.37 ms, sys: 1.06 ms, total: 9.44 ms
Wall time: 311 ms


'\n\nMost spiders have eight eyes.'

In [13]:
%%time
LOGIC_QUESTION_FORM_2 = "Is it true that the absence of a proof equates the proof of an absence?"
# Switching to the other question again
llm(LOGIC_QUESTION_FORM_2)

CPU times: user 22.4 ms, sys: 2.42 ms, total: 24.9 ms
Wall time: 518 ms


'\n\nNo, absence of proof is not the same as proof of absence.'

## Additional options

When creating the semantic cache, you can specify a few other options such as the metric used to calculate the similarity and the number of entries to retrieve in the ANN step (i.e. those on which the exact requested metric is computed for the final filtering). Here is an example which uses the L2 metric:

In [14]:
anotherCassSemanticCache = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    distance_metric='l2',
    score_threshold=0.4,
    num_rows_to_fetch=12,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

This cache builds on the same database table as the previous one, as can be seen e.g. with:

In [15]:
lookup = anotherCassSemanticCache.lookup_with_id_through_llm(
    LOGIC_QUESTION_FORM_2,
    llm,
)
if lookup:
    docId, response = lookup
    print(docId)
    print(response)
else:
    print('No match.')

77add13036bcaa23c74ebf2ab2c56441
[Generation(text='\n\nNo, absence of proof is not the same as proof of absence.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nThere is a difference between absence of proof and proof of absence. Absence of proof means that there is no evidence to support a claim. Proof of absence means that there is evidence that disproves a claim.', generation_info={'finish_reason': 'stop', 'logprobs': None})]


## Stale entry control

### Time-To-Live (TTL)

You can configure a time-to-live property of the cache, with the effect of automatic eviction of cached entries after a certain time.

Setting `langchain.llm_cache` to the following will have the effect that entries vanish in an hour:

In [16]:
cacheWithTTL = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    ttl_seconds=15,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

### Manual cache eviction

Alternatively, you can invalidate individual entries one at a time, just like you saw for the exact-match `CassandraCache` cache.

But this is an index based on sentence similarity, so this time the procedure has two steps: **first**, a lookup to find the _id_ of the matching document:

In [17]:
lookup = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_1, llm)
if lookup:
    docId, response = lookup
    print(docId)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


_you can see that querying for another form for the "same" question will result in the same id:_

In [18]:
lookup2 = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_2, llm)
if lookup:
    docId2, response2 = lookup2
    print(docId2)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


and **second**, the document id is used in the actual cache eviction (again, you have to additionally provide the LLM):

In [19]:
cassSemanticCache.delete_by_document_id_through_llm(docId, llm)

As a check, try asking that question again:

In [20]:
%%time
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 1.23 s


'\n\nMost spiders have eight eyes, but some have fewer or more.'

### Whole-cache deletion

Lastly, as you have seen earlier, you can empty the cache entirely, for a given LLM, with:

In [21]:
cassSemanticCache.clear_through_llm(llm=llm)